# HW3 - Brian Flores

Ch4 and Ch5
  
---

## CH4 Subqueries

## Q7

Write a query that returns customers, who ordered product 12

**Tables involved**: TSQLV6 database, Customers, Orders and OrderDetails tables

**Desired output:**

```

custid      companyname

----------- ----------------------------------------
48          Customer DVFMB
39          Customer GLLAG
71          Customer LCOUJ
65          Customer NYUHS
44          Customer OXFRU
51          Customer PVDZC
86          Customer SNXOJ
20          Customer THHDP
90          Customer XBBVR
46          Customer XPNIK
31          Customer YJCBX
87          Customer ZHYOS

```

(12 rows affected)

* * *

## **Proposition, Table, Columns and the Predicate.**

**Proposition:** Retrieve customer IDs and company names for customers who have placed orders with a product ID of 12.

**Tables:** `[Sales].[Customers]` AS `C`, `[SALES].[Orders]` AS `O`, `[SALES].[OrderDetails]` AS `OD`

**Columns:** `C.custid`, `C.companyname`

**Predicate:** There exists rows such that:

- Rows in table `C`, that are logically connected to an Order in `O` this is done on equality of customer ids i.e. `C.custid` is equal to `O.custid`:
    
    - Next, there exists rows in `OD`,that are logically connected and are equal to a product ID of 12 and through a order id from `O`. In other words the following holds:
        - `OD.orderid` is equal to `O.orderid`
        - `OD.productid` is equal to 12

**_Written in (Minor) collaboration with ChatGPT from OpenAI to improve understanding and assist with the explanation of the query._**

In [ ]:
-- Q7

-- Original Query for TSQLV6

USE TSQLV6;

SELECT C.custid, C.companyname
FROM [Sales].[Customers] AS C
WHERE EXISTS 
    (SELECT *
    FROM [Sales].[Orders] AS O 
    WHERE C.custid = O.custid  
       AND EXISTS                     
        (SELECT * 
        FROM [Sales].[OrderDetails] AS OD 
        WHERE OD.orderid = O.orderid         
           AND OD.productid = 12));

-- New Query for Northwinds

USE Northwinds2022TSQLV7;

SELECT C.CustomerId, C.CustomerCompanyName
FROM [Sales].[Customer] AS C
WHERE EXISTS
    (SELECT *
    FROM [Sales].[Order] AS O
    WHERE C.CustomerId = O.CustomerId
        AND EXISTS
        (SELECT *
        FROM [Sales].[OrderDetail] AS OD
        WHERE OD.OrderId = O.OrderId
            AND OD.ProductId = 12));


## Q8

Write a query that calculates a running total qty for each customer and month using subqueries

**Views involved:** TSQLV6 database, Sales.CustOrders view

**Desired output:**

```
custid      ordermonth  qty         runqty
----------- ----------- ----------- -----------
1           2021-08-01  38          38
1           2021-10-01  41          79
1           2022-01-01  17          96
1           2022-03-01  18          114
1           2022-04-01  60          174
2           2020-09-01  6           6
2           2021-08-01  18          24
2           2021-11-01  10          34
2           2022-03-01  29          63
3           2020-11-01  24          24
3           2021-04-01  30          54
3           2021-05-01  80          134
3           2021-06-01  83          217
3           2021-09-01  102         319
3           2022-01-01  40          359
...

```

(636 rows affected)

* * *

## **Proposition, Table, Columns and the Predicate.**

**Proposition:** Retrieve exactly those customer IDs in the view **\[Sales\].\[CustOrders\] view**, order dates, and quantities along with a running total quantity (for each respective customer). The running total is the sum of quantities for each customer up to the current order date.

**Tables:** `[Sales].[CustOrders]` AS `CO1`, `[Sales].[CustOrders]` AS `CO2`

**Columns:** `CO1.custid`, `CO1.ordermonth`, `CO1.qty`, (A running aggregrate) AS `runqty`

**Predicate:** There exists rows such that:

- For each (**customer**) row in `CO1`, there is a subquery that calculates the running total (`runqty`). This subquery is logically connected to the main query by the condition:
    - `CO2.ordermonth` is less than or equal to `CO1.ordermonth` (ensuring we sum quantities only up to the current order month).
    - `CO2.custid` is equal to `CO1.custid` (ensuring we calculate the running total for the specific customer).

**Take away:**

Using Ben-Gans subquery strategy "running aggregrates" we can get a join like behavior for two identical tables. We do this by specifying the predicate in the subquery and then taking the sum.

The final result is ordered by customer ID (`CO1.custid`) and order month (`CO1.ordermonth`).

 **_Written in collaboration with ChatGPT from OpenAI to improve understanding and assist with the explanation of the query._**

In [ ]:
-- Q8

-- Original Query for TSQLV6

USE TSQLV6;

SELECT CO1.custid, CO1.ordermonth, CO1.qty,
    (SELECT SUM(CO2.qty)
    FROM [Sales].[CustOrders] AS CO2
    WHERE CO2.ordermonth <= CO1.ordermonth 
        AND CO2.custid = CO1.custid) AS runqty
FROM [Sales].[CustOrders] AS CO1
ORDER BY CO1.custid, CO1.ordermonth;

-- New Query for Northwinds
-- There is no view [Sales].[CustOrders] for NorthWinds. The bottom query asserts this 

USE Northwinds2022TSQLV7; 

SELECT 
    schema_name(schema_id) AS schema_name,
    name AS view_name
FROM 
    sys.views;


# Propositions

Translating queries to problems.

## NULL TROUBLE

In this section we go over how bugs can be introdcued when we involve nulls with subqueries.

### N.T 1

Assuming there is no null entry for customer id in the `[Sales].[Order]` return all customers who did not make a purchase.

**Tables Involved:**`[Sales].[Customer]`, `[Sales].[Order]`  

**Constraints**: use `IN`

**Takeaways**: The IN operator works like a disjunction of many predicates (x=1 OR y=1 OR ... )

In [ ]:
-- N.T 1

-- Original Query for TSQLV6
USE TSQLV6;

SELECT custid, companyname
FROM [Sales].[Customers]
WHERE custid NOT IN (SELECT O.custid
                    FROM [Sales].[Orders] AS O);

-- Original Query for NorthWinds
USE Northwinds2022TSQLV7;

SELECT CustomerId, CustomerCompanyName
FROM [Sales].[Customer]
WHERE CustomerId NOT IN (SELECT O.CustomerId
                        FROM [Sales].[Order] AS O);


## NT.2

Insert a null entry into `[Sales].[Order]`. Where `NULL` for `custid`.`1` for `empid`. `'20220212'` for `orderdate`, `requireddate`, and `shippeddate`. `1` for `shipperid`. `123.00` for `freight`. `N'abc'` for string values in the remaining columns (`shipname`, `shipaddress`, `shipcity`, `shipregion`, `shippostalcode`, `shipcountry`)

**Tables Involved:** `[Sales].[Order]`, 

Make sure to clean up where necessary.

In [ ]:
-- N.T 2

USE TSQLV6;

-- PRE Cleanup
DELETE FROM [Sales].[Orders] WHERE custid IS NULL;

INSERT INTO [Sales].[Orders]
  (custid, empid, orderdate, requireddate, shippeddate, shipperid,
   freight, shipname, shipaddress, shipcity, shipregion,
   shippostalcode, shipcountry)
  VALUES(NULL, 1, '20220212', '20220212',
         '20220212', 1, 123.00, N'abc', N'abc', N'abc',
         N'abc', N'abc', N'abc');


USE Northwinds2022TSQLV7;

-- PRE Cleanup
DELETE FROM [Sales].[Order] WHERE CustomerId IS NULL;

INSERT INTO [Sales].[Order]
  (CustomerId, EmployeeId, OrderDate, RequiredDate, ShipToDate, ShipperId,
   Freight, ShipToName, ShipToAddress, ShipToCity, ShipToRegion,
   ShipToPostalCode, ShipToCountry)
  VALUES(NULL, 1, '20220212', '20220212',
         '20220212', 1, 123.00, N'abc', N'abc', N'abc',
         N'abc', N'abc', N'abc');

### N.T 3

Assuming there is now a Null Entry in `[Sales].[Customers]` experiment with `IN` to return an empty set when using the same query in NT.1. Explain why this occurs. \*\*Make sure to clean up after exploring this \*\* **Tables Involved:**`[Sales].[Customers]`

**Tables Involved:**`[Sales].[Customer]`, `[Sales].[Order]`

**Constraints**: use `IN`

**Takeaways**: The IN operator works like a disjunction of many predicates (x=1 OR y=1 OR ... ). And `IN` wors on 3 predicate logic meaning working with nulls will reduce a predicate to UNKNOWN. Therefore the IN operator will not return anything

In [ ]:
-- N.T 3

USE TSQLV6;

-- Return the empty set
SELECT custid, companyname
FROM [Sales].[Customers]
WHERE custid NOT IN(SELECT O.custid
                    FROM [Sales].[Orders] AS O);

USE Northwinds2022TSQLV7;

-- return the empty set
SELECT CustomerId, CustomerCompanyName
FROM [Sales].[Customer]
WHERE CustomerId NOT IN(SELECT O.CustomerId
                    FROM [Sales].[Order] AS O);


### N.T 4

Redo the same query as before using `IN` but catch and discard any null rows you observe in the subquery. Now in this case return all customers who did not make a purchase Within the `[Sales].[Customers]` table. This should not result in the empty set.

**Tables Involved:**`[Sales].[Customer]`, `[Sales].[Order]`

**Constraints**: use `IN`, use `IS NOT NULL`

**Takeaways**: The IN operator works like a disjunction of many predicates (x=1 OR y=1 OR ... ). And `IN` works on 3 predicate logic purposely excluding (via `IS NOT NULL`) it in the subquery can help us avoid the empty set if we use `IN`.


In [ ]:
-- N.T 4

USE TSQLV6;

-- Exclude NULLs explicitly
SELECT custid, companyname
FROM [Sales].[Customers]
WHERE custid NOT IN (SELECT O.custid 
                    FROM [Sales].[Orders] AS O
                    WHERE O.custid IS NOT NULL);

USE Northwinds2022TSQLV7;

-- Exclude NULLs explicitly
SELECT CustomerId, CustomerCompanyName
FROM [Sales].[Customer]
WHERE CustomerId NOT IN (SELECT O.CustomerId
                    FROM [Sales].[Order] AS O
                    WHERE O.CustomerId IS NOT NULL);


### N.T 5

When getting customers who did not place an order use `EXISTS` to filter out rows from a subquery.

**Tables Involved:**`[Sales].[Customers]`, `[Sales].[Order]`

**Constraints**: use `EXISTS`

**Takeaways**: `EXISTS` works on 2 predicate logic. It takes in a subquery and outputs a boolean value whether at least just one row was returned from a subquery.



In [ ]:
-- N.T 5

USE TSQLV6;

SELECT custid, companyname
FROM [Sales].[Customers] AS C
WHERE NOT EXISTS
  (SELECT * 
   FROM [Sales].[Orders] AS O
   WHERE O.custid = C.custid);

USE Northwinds2022TSQLV7;

SELECT CustomerId, CustomerCompanyName
FROM [Sales].[Customer] AS C
WHERE NOT EXISTS
  (SELECT * 
   FROM [Sales].[Order] AS O
   WHERE O.CustomerId = C.CustomerId);

In [ ]:
-- POST CLEANUP

USE TSQLV6;

DELETE FROM Sales.Orders WHERE custid IS NULL;


USE Northwinds2022TSQLV7;

-- PRE Cleanup
DELETE FROM [Sales].[Order] WHERE CustomerId IS NULL;

---
## CH5 Table Expressions


## **Q6-1**

Create an inline function that accepts as inputs
a supplier id (@supid AS INT), 
and a requested number of products (@n AS INT)
The function should return @n products with the highest unit prices
that are supplied by the given supplier id

**Tables involved**: Production.Products

desired output when issuing the following query:
```sql
SELECT * FROM Production.TopProducts(5, 2)
-- When done Execute this for clean up
DROP FUNCTION IF EXISTS Production.TopProducts;

```

```

productid  productname   unitprice
--------   -----------   ----------
12        Product OSFNS     38.00
11        Product QMVUN     21.00

```
(2 rows affected)

# **Proposition, Table, Columns and the Predicate.**


**Proposition:**
Create a **inline table-valued function** named `Production.TopProducts` that takes two parameters, `@supid` (Supplier ID) and `@n` (number of top products to retrieve). The function returns a table with **columns** `ProductId`, `ProductName`, and `UnitPrice` for the top `@n` products from the `Production.Product` table where the `SupplierId` matches the provided `@supid`. The results are ordered by `UnitPrice` in descending order (Top orders are first).

**Tables:**
`[Production].[Product]`

**Columns:**
`ProductId`, `ProductName`, `UnitPrice`

**Predicate:**
Return rows such that:

- Rows in table `Production.Product`, where:
  - `SupplierId` is equal to the input parameter `@supid`.
  - The result is limited to the top `@n` rows based on the intensity of `unitprice`.
  

**Explanation:**

The query creates or alters a function named `Production.TopProducts` that returns a table with product information for the top `@n` products associated with a specified supplier (`@supid`). The result set is limited to the top `@n` products based on their `ProductID`, and the results are ordered by `UnitPrice` in descending order.

**AI Written in collaboration with ChatGPT from OpenAI to improve understanding and assist with the explanation of the query.**

In [ ]:
-- Q6.1

USE TSQLV6;

GO
CREATE OR ALTER FUNCTION Production.TopProducts
  (@supid AS INT, @n AS INT) RETURNS TABLE
AS
RETURN
  SELECT TOP(@n) productid, productname, unitprice
  FROM [Production].[Products] 
  WHERE supplierid = @supid
  ORDER BY unitprice DESC;
GO

USE Northwinds2022TSQLV7;

GO
CREATE OR ALTER FUNCTION Production.TopProducts
  (@supid AS INT, @n AS INT) RETURNS TABLE
AS
RETURN
  SELECT TOP(@n) ProductId, ProductName, UnitPrice
  FROM [Production].[Product] 
  WHERE SupplierId = @supid
  ORDER BY UnitPrice DESC;
GO

In [ ]:
-- TEST 

USE TSQLV6;
SELECT * FROM [Production].[TopProducts](5, 2);

USE Northwinds2022TSQLV7;
SELECT * FROM [Production].[TopProducts](5, 2);


### **Q6-2**
Using the CROSS APPLY operator
and the function you created in exercise 6-1,
return, for each supplier, the two most expensive products

**Tables involved**: Production.Products


```
supplierid  companyname     productid   productname     unitprice
----------- --------------- ----------- --------------- ----------
8           Supplier BWGYE  20          Product QHFFP   81.00
8           Supplier BWGYE  68          Product TBTBL   12.50
20          Supplier CIYNM  43          Product ZZZHR   46.00
20          Supplier CIYNM  44          Product VJIEO   19.45
23          Supplier ELCRN  49          Product FPYPN   20.00
23          Supplier ELCRN  76          Product JYGFE   18.00
5           Supplier EQPNC  12          Product OSFNS   38.00
5           Supplier EQPNC  11          Product QMVUN   21.00
...

```
(55 rows affected)



# **Proposition, Table, Columns and the Predicate.**

**Proposition:**

For each unique supplier id in the **table** `[Production].[Supplier]` get thier top 2 most products. Do this using a **Inline Table Value Function** `Production.TopProducts`. Output the supplierID, companyName, productId, productName and unitprice for the product. Achieve this behavior using `CROSS APPLY`.

**Tables:**
`[Production].[Supplier]` AS `S`

**Columns:**
`S.SupplierId`, `S.SupplierCompanyName`, `T.ProductId`, `T.ProductName`, `T.UnitPrice`

**Predicate:**

Obtain a summary where:
- Rows in `Production.Supplier` AS ` S` table, representing suppliers.
- For each row in `S`, apply the `Production.TopProducts` function using `CROSS APPLY` to get the top 2 products for the corresponding supplier.


**Explanation:**

The query retrieves information about suppliers via a unique (`S.SupplierId`) along with details about the top 2 products for each supplier. This is achieved by applying the `Production.TopProducts` function to each supplier using `CROSS APPLY`. The function takes the Supplier ID (`S.SupplierId`) and the parameter `n` to retrieve the top n products based on unit price. The result set includes columns from both the `Production.Supplier` table (`S`) and the result of the function (`T`), providing a comprehensive view of the top products for each supplier.

**AI Written in (MINOR) collaboration with ChatGPT from OpenAI to improve understanding and assist with the explanation of the query.**

In [ ]:
-- Q6.2

USE TSQLV6;

SELECT S.supplierid, S.companyname, T.productid, T.productname, T.unitprice
FROM [Production].[Suppliers] AS S
    CROSS APPLY [Production].[TopProducts](S.supplierid, 2) AS T;

USE Northwinds2022TSQLV7;

SELECT S.SupplierId, S.SupplierCompanyName, T.Productid, T.ProductName, T.UnitPrice
FROM [Production].[Supplier] AS S
    CROSS APPLY [Production].[TopProducts](S.SupplierId, 2) AS T;

In [ ]:
-- CLEANUP for 6.1

USE TSQLV6;
DROP FUNCTION IF EXISTS Production.TopProducts;

USE Northwinds2022TSQLV7;
DROP FUNCTION IF EXISTS Production.TopProducts;

# Propositions

Translating queries to problems.

---
## **NESTING**

**N.1**

Using a nested derived table first create a table from `[Sales].[Order]` exporting only years and their respective customer ids. Using that same derived table create another derived table that uses the previous derived table now group relevant order years and count the unique ids within a year. Lastly use that table to filter out years that had more than 70 or more customers.

In othewords return exactly those rows for every year have more customers than 70. return the year and the number of customers.

**Tables Involved:** `[Sales].[Order]`

**Discuss:** Is the trade off in column aliasing  (A benefit of table expressions) worth it than a method that require multiple function calls?
 

In [ ]:
-- N.1

USE TSQLV6;

SELECT orderyear, numcusts
FROM (SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
      FROM (SELECT YEAR(orderdate) AS orderyear, custid
            FROM [Sales].[Orders]) AS D1
      GROUP BY orderyear) AS D2
WHERE numcusts > 70;

USE Northwinds2022TSQLV7;

SELECT OrderYear, NumCusts
FROM (SELECT OrderYear, COUNT(DISTINCT CustomerId) AS NumCusts
    FROM (SELECT YEAR(OrderDate) AS OrderYear, CustomerId
        FROM [Sales].[Order]) AS D1
    GROUP BY OrderYear) AS D2
WHERE NumCusts > 70;


**N.2**


Return order years and he number of customers handled in each year but only for years that have more than 70 unique customers. Do this without any table expressions output the orderyear and number of customers.

**Tables Involved:** `[Sales].[Order]`



In [ ]:
-- N.2

USE TSQLV6;

SELECT YEAR(orderdate) AS orderyear, COUNT(DISTINCT custid) AS numcusts
FROM [Sales].[Orders]
GROUP BY YEAR(orderdate)
HAVING COUNT(DISTINCT custid) > 70;


USE Northwinds2022TSQLV7;

SELECT YEAR(OrderDate) AS OrderYear, COUNT(DISTINCT CustomerId) AS NumCusts
FROM [Sales].[Order]
GROUP BY YEAR(OrderDate)
HAVING COUNT(DISTINCT CustomerId) > 70;

---
## **Using Arguments in CTEs**

**UAC.1**

Declare a variable labeled `@empid` with the value of 3. Use a common table expression to get all orders from the variable `@empid`. 

That CTE should output the years and customerids related. Use the CTE to display all orderyears with their respective unique customer years. 

**Tables Involved:** `[Sales].[Order]`


**Disscuss:** What is the difference between derived tables, CTEs and Views. Can you use arguments in Derived table expressions? Why choose one over the other


In [ ]:
-- UAC.1

USE TSQLV6;

DECLARE @empid AS INT = 3;

WITH C AS
(
  SELECT YEAR(orderdate) AS orderyear, custid
  FROM [Sales].[Orders]
  WHERE empid = @empid
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM [C]
GROUP BY orderyear;
GO

USE Northwinds2022TSQLV7;

DECLARE @empid AS INT = 3;

WITH C AS
(
    SELECT YEAR(OrderDate) AS OrderYear, CustomerId
    FROM [Sales].[Order]
    WHERE EmployeeId = @empid
)
SELECT OrderYear, COUNT(DISTINCT CustomerId) AS NumCusts
FROM [C]
GROUP BY OrderYear;
GO



---
## **Recursive CTEs**

**RC.1**

Find all **employees who are below Don Funk** repeat this pattern until no more employees are, this includes Don Funk. In other words find everyone who is below and connected to Don Funk. Display the employee id manager id their first and last name as well as their level respective to Don Funk who can be considered level 1.

**Tables Involved:** `[HR].[Employee]`



In [ ]:
-- R.C 1

USE TSQLV6;

WITH [EmpsCTE] AS
(
  SELECT empid, mgrid, firstname, lastname, 1 AS lvl
  FROM [HR].[Employees]
  WHERE empid = 2
  
  UNION ALL
  
  SELECT C.empid, C.mgrid, C.firstname, C.lastname, lvl + 1
  FROM [EmpsCTE] AS P
    INNER JOIN [HR].[Employees] AS C
      ON C.mgrid = P.empid
)
SELECT empid, mgrid, firstname, lastname, lvl
FROM [EmpsCTE];
GO

USE Northwinds2022TSQLV7;

WITH EmpsCTE AS 
 (
    SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName, 1 as lvl
    FROM [HumanResources].[Employee] 
    WHERE EmployeeId = 2

    UNION ALL

    SELECT C.EmployeeId, C.EmployeeManagerId, C.EmployeeFirstName, C.EmployeeLastName, lvl + 1
    FROM [EmpsCTE] AS P
      INNER JOIN [HumanResources].[Employee] AS C
        ON C.EmployeeManagerId = p.EmployeeId
 )
SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName, lvl
FROM [EmpsCTE];

GO



In [ ]:
-- Shows a step by step how the recrusive query takes place

USE TSQLV6;

-- Show the anchor element
SELECT C.empid, C.mgrid, C.firstname, C.lastname
FROM HR.Employees AS C
WHERE C.empid = 2;

--- Comment out 1 by 1 the conditions show it uses the previous set to get next rows
SELECT C.empid, C.mgrid, C.firstname, C.lastname, P.empid, P.mgrid, P.firstname, P.lastname
  FROM HR.Employees AS P
    INNER JOIN HR.Employees AS C
      ON C.mgrid = P.empid
    WHERE P.empid = 2 -- or P.empid = 3 -- or P.empid = 5;

---
## **Encryption**

**E.1**

Create a custom view that uses `[Sales].[Customer]` which only has American customers. First create the view without any encryption test that you can see it and then use Encryption assert that you can not see it.

**Tables Involved:** `[Sales].[Customer]`



In [ ]:
-- Use this Query to switch databases
-- E.1.0

USE Northwinds2022TSQLV7;


In [ ]:
CREATE OR ALTER VIEW [Sales].[USACusts]
AS
SELECT CustomerId, CustomerCompanyName, CustomerContactName,CustomerContactTitle, CustomerAddress,
    CustomerCity, CustomerRegion, CustomerPostalCode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM [Sales].[Customer]
WHERE CustomerCountry = N'USA';
GO

-- E.1.1
-- Create statement must be the first thing to be executed.

SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));


In [ ]:
CREATE OR ALTER VIEW [Sales].[USACusts] WITH ENCRYPTION
AS
SELECT CustomerId, CustomerCompanyName, CustomerContactName,CustomerContactTitle, CustomerAddress,
    CustomerCity, CustomerRegion, CustomerPostalCode, CustomerCountry, CustomerPhoneNumber, CustomerFaxNumber
FROM [Sales].[Customer]
WHERE CustomerCountry = N'USA';
GO

-- E.1.1
-- Create statement must be the first thing to be executed. If you execute this you get null

SELECT OBJECT_DEFINITION(OBJECT_ID('Sales.USACusts'));


In [ ]:
-- E.1.2 See how it is stored as a procedure and also check if its encrypted
EXEC sp_helptext 'Sales.USACusts';
GO

---
## **Inline User Defined Functions**

**IUDF.1**

Create a custom function that filters for specific rows having the same customer id in the `[Sales].[Customer]` table. the argument should be an int represening the customer id and should be called `[dbo].[GetCustOrders]`.

**Tables Involved:** `[Sales].[Customer]`

Make sure you test in a single table query like so.

```sql
SELECT orderid, custid
FROM dbo.GetCustOrders(1) AS O;
```

Next Test you can use this function on a join like this.

```sql
SELECT O.orderid, O.custid, OD.productid, OD.qty
FROM dbo.GetCustOrders(1) AS O
  INNER JOIN Sales.OrderDetails AS OD
    ON O.orderid = OD.orderid;
```

The desired output should look like this


|orderid|custid|productid|qty|
|---|---|---|---|
|10643|1|28|15|
|10643|1|39|21|
|10643|1|46|2|
|10692|1|63|20|
...






In [ ]:
-- IUDF.1.0

USE TSQLV6;

GO
CREATE OR ALTER FUNCTION dbo.GetCustOrders
  (@cid AS INT) RETURNS TABLE
AS
RETURN
  SELECT orderid, custid, empid, orderdate, requireddate,
    shippeddate, shipperid, freight, shipname, shipaddress, shipcity,
    shipregion, shippostalcode, shipcountry
  FROM [Sales].[Orders]
  WHERE custid = @cid;
GO

USE Northwinds2022TSQLV7;

GO
CREATE OR ALTER FUNCTION dbo.GetCustOrders
  (@cid AS INT) RETURNS TABLE
AS
RETURN
  SELECT OrderId, CustomerId, EmployeeId, OrderDate, RequiredDate,
    ShipToDate, ShipperId, Freight, ShipToName, ShipToAddress, ShipToCity,
    ShipToRegion, ShipToPostalCode, ShipToCountry
  FROM [Sales].[Order]
  WHERE CustomerId = @cid;
GO


In [ ]:
-- IUDF.1.1

USE TSQLV6;

-- Test Functions
SELECT orderid, custid
FROM dbo.GetCustOrders(1) AS O;

SELECT O.orderid, O.custid, OD.productid, OD.qty
FROM dbo.GetCustOrders(1) AS O
  INNER JOIN Sales.OrderDetails AS OD
    ON O.orderid = OD.orderid;
GO


USE Northwinds2022TSQLV7;

SELECT OrderId, CustomerId
FROM dbo.GetCustOrders(1) AS O;

SELECT O.OrderId, O.CustomerId, OD.productid, OD.Quantity
FROM dbo.GetCustOrders(1) AS O
  INNER JOIN [Sales].[OrderDetail] AS OD
    ON O.orderid = OD.orderid;
GO


In [ ]:
-- IUDF.1.2
-- Cleanup

USE TSQLV6;

DROP FUNCTION IF EXISTS dbo.GetCustOrders;
GO

USE Northwinds2022TSQLV7;

DROP FUNCTION IF EXISTS dbo.GetCustOrders;
GO